In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
output_path = "/Users/heytitle/projects/jourd-politics/src/data"

def prepend_path(s):
    return "%s/%s" % (output_path, s)

In [3]:
df_projects = pd.read_csv("../data/2019-local-administrative-organization-projects.csv")

/Users/heytitle/projects/jour-d/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
len(df_projects)

1085388

# Global Statistics

In [5]:
global_stats = dict(
    totalProjects = len(df_projects), 
    sumProjectValue = df_projects.sum_price_agree.sum()
)



In [6]:
def is_civil_proj(x):
    include_key_words = ['ลูกรัง','ลาดยาง','สร้างถนน','ซ่อมแซมถนน','ซ่อมผิวจราจร','ท่อระบายน้ำ','หินคลุก','คสล',\
                 'คอนกรีตเสริมเหล็ก','ซ่อมแซมถนน','แอสฟัล','ประปา','บาดาล','บาดาล','สะพาน','ตลิ่ง']
    
    bool_result = False
    
    for a_key_word in include_key_words:
        bool_result |= (a_key_word in x)

    return bool_result

df_civil_projects = df_projects[df_projects.project_name.apply(is_civil_proj)]

global_stats['totalCivilProjects'] = len(df_civil_projects)
global_stats['sumCivilProjectValue'] = df_civil_projects.sum_price_agree.sum()

print(global_stats)

{'totalProjects': 1085388, 'sumProjectValue': 112169484402.0, 'totalCivilProjects': 151917, 'sumCivilProjectValue': 49962973805.0}


In [7]:
with open(prepend_path("global_stats.json"), "w") as fh:
    json.dump(global_stats, fh)

# Procurement Method Statistics

In [8]:
df_proc_med_stats = df_projects.groupby("purchase_method_name")["sum_price_agree"]


df_proc_med_stats_merged = df_proc_med_stats.count().reset_index().rename(columns={'sum_price_agree':'totalProjects'})\
    .merge(df_proc_med_stats.sum().reset_index().rename(columns={'sum_price_agree':'totalProjectValue'}), on="purchase_method_name")


with open(prepend_path("procurement_methods_stats.json"), "w") as fh:
    json.dump(df_proc_med_stats_merged.to_dict("record"), fh, ensure_ascii=False)

In [9]:
df_proc_med_stats.count().reset_index()

,purchase_method_name,sum_price_agree
0,กรณีพิเศษ,48
1,คัดเลือก,2514
2,จ้างที่ปรึกษาโดยวิธีคัดเลือก,8
3,จ้างที่ปรึกษาโดยวิธีตกลง,1
4,จ้างที่ปรึกษาโดยวิธีประกาศเชิญชวนทั่วไป,5
5,จ้างที่ปรึกษาโดยวิธีเฉพาะเจาะจง,231
6,ตกลงราคา,1394
7,ตลาดอิเล็กทรอนิกส์ (e-market),28
8,ประกวดราคา,2
9,ประกวดราคาด้วยวิธีการทางอิเล็กทรอนิกส์,743


# Authority Statistics

In [10]:
df_local_author_raw = df_projects
df_local_author_stats = df_local_author_raw.groupby(["dept_name", "province"])["sum_price_agree"]

df_local_author_stats.count().reset_index().rename(columns={'sum_price_agree':'totalProjects'})


df_local_author_stats = df_local_author_stats.count().reset_index().rename(columns={'sum_price_agree':'totalProjects'})\
    .merge(df_local_author_stats.sum().reset_index().rename(columns={'sum_price_agree':'totalProjectValue'}), on=['dept_name', 'province'])

df_local_author_stats[:10]


local_author_stats_data = dict()

for r in df_local_author_stats.to_dict("record"):
    r['methodStats'] = dict()
    local_author_stats_data['%s-%s' % (r['dept_name'], r['province'])] = r

In [11]:
df_local_author_method_stats = df_local_author_raw.groupby(["dept_name", "province", "purchase_method_name"])["sum_price_agree"]

df_local_author_method_stats_merged = df_local_author_method_stats.count().reset_index().rename(columns={'sum_price_agree':'totalProjects'}) \
    .merge(df_local_author_method_stats.sum().reset_index().rename(columns={'sum_price_agree':'totalProjectValue'}), on=['dept_name', 'province', 'purchase_method_name'])

for r in df_local_author_method_stats_merged.to_dict("record"):
    key = '%s-%s' % (r['dept_name'], r['province'])
    local_author_stats_data[key]['methodStats'][r['purchase_method_name']] = dict(
        count=r['totalProjects'], value=r['totalProjectValue']
    )
    if r['purchase_method_name'] == "เฉพาะเจาะจง":
        local_author_stats_data[key]['specialProcurementCount'] = r['totalProjects']
    else: 
        local_author_stats_data[key]['specialProcurementCount'] = 0

In [12]:
with open(prepend_path("local_authority_stats.json"), "w") as fh:
    sorted_local_author_stats_data =  sorted(local_author_stats_data.values(), key = lambda x: -x['specialProcurementCount'])
    
    json.dump(sorted_local_author_stats_data[:100], fh, ensure_ascii=False)

# Company Statistics

In [32]:
thresdhold = 1e8

In [33]:
filter_names = [
#     "สหกรณ์โคนมวังน้ำเย็น จำกัด"
    "something"
]

df_projects_have_comp = df_projects[~df_projects.corporate_name.isna()]
df_projects_have_comp = df_projects_have_comp[~df_projects_have_comp.corporate_name.str.contains("|".join(filter_names))]

In [34]:
df_company_stats = df_projects_have_comp.groupby(["corporate_name"])["sum_price_agree"]

df_company_stats_merged = df_company_stats.count().reset_index().rename(columns={'sum_price_agree':'totalProjects'}) \
    .merge(df_company_stats.sum().reset_index().rename(columns={'sum_price_agree':'totalProjectValue'}), on=['corporate_name'])


In [35]:
df_company_stats_selected = df_company_stats_merged[df_company_stats_merged.totalProjectValue > thresdhold]

In [36]:
len(df_company_stats_selected)

54

In [18]:
company_stats_data = dict()

for r in df_company_stats_selected.to_dict("record"):
    r['methodStats'] = dict()
    r['regionCounts'] = []
    r['projectInsights'] = []
    company_stats_data[r['corporate_name']] = r

In [19]:
df_project_join_selected_company = df_projects.merge(df_company_stats_selected, on=['corporate_name'], how="left")
df_project_join_selected_company = df_project_join_selected_company[df_project_join_selected_company.totalProjects > 0]

In [20]:
df_company_method_stats = df_project_join_selected_company.groupby(["corporate_name", "purchase_method_name"])["sum_price_agree"]

df_company_method_stats_merged = df_company_method_stats.count().reset_index().rename(columns={'sum_price_agree':'totalProjects'}) \
    .merge(df_company_method_stats.sum().reset_index().rename(columns={'sum_price_agree':'totalProjectValue'}), on=['corporate_name', 'purchase_method_name'])

for r in df_company_method_stats_merged.to_dict("record"):
    key = r['corporate_name']
    
    company_stats_data[key]['methodStats'][r['purchase_method_name']] = dict(
        count=r['totalProjects'], value=r['totalProjectValue']
    )

In [21]:
df_company_region_stats = df_project_join_selected_company.groupby(["corporate_name", "region"])["sum_price_agree"]

df_company_region_stats_flattened = df_company_region_stats.count().reset_index().rename(columns={'sum_price_agree': 'count'})

for r in df_company_region_stats_flattened.to_dict("record"):
    company_stats_data[r['corporate_name']]['regionCounts'].append(dict(name=r['region'], count=r['count']))
    

In [22]:
df_tins = df_project_join_selected_company[['corporate_name', 'winner_tin']].drop_duplicates()

name_to_tins = dict(zip(df_tins['corporate_name'].values, df_tins['winner_tin'].values))

In [23]:
df_company_project_insight = df_project_join_selected_company.groupby(["corporate_name", "province", "purchase_method_name"])["sum_price_agree"]

df_company_project_insight_flattened = df_company_project_insight.sum().reset_index().rename(columns={'sum_price_agree': 'value'})

for r in df_company_project_insight_flattened.to_dict("record"):
    company_stats_data[r['corporate_name']]['projectInsights'].append(
        dict(source=r['purchase_method_name'], target=r['province'], value=r['value'])
    )

In [24]:
with open(prepend_path("company_stats.json"), "w") as fh:
    data = []
    
    for d in company_stats_data.values():
        primaryRegion = sorted(d['regionCounts'], key=lambda x: x['count'])[-1]['name']
        d['primaryRegion'] = primaryRegion
        d['tin'] = name_to_tins[d['corporate_name']]
        
        data.append(d)
    json.dump(data, fh, ensure_ascii=False)

In [25]:
len(df_company_project_insight)

923